In [2]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 MB 1.9 MB/s eta 0:00:00m eta 0:00:010:00:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 KB 688.8 kB/s eta 0:00:00 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-4.0.0-py2.py3-none-any.whl size=434741258 sha256=5cbceed13aae1c3f6967f4b8c9ff71e34fffa9e0667a6d932fc5ba19448db945
  Stored in directory: /home/exbom/.cache/pip/wheels/62/69/eb/eef3014e40bbcff88f1d6dd762baebf6bf5d0266ba57be1ef8
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum

In [17]:
#iniciar um sessão Spark
spark = SparkSession.builder.appName("Demo").getOrCreate()

#Leitura do arquivo csv
df_vendas = spark.read.csv("vendas.csv", header=True, inferSchema=True)

#Leitura do arquivo Json
df_clientes = spark.read.json("clientes.json")

In [18]:
#Exibir os esquemas dos DataFrames
print("Equema do DataFrame de vendas:")
df_vendas.printSchema()

print("\nEsquema do DataFrame de clientes:")
df_clientes.printSchema()

#Exibição dos dados:
print("Dados de vendas:")
df_vendas.show()

print("\nDados de clientes:")
df_clientes.show()

Equema do DataFrame de vendas:
root
 |-- ID_Venda: integer (nullable = true)
 |-- Data_Venda: date (nullable = true)
 |-- Cliente: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Preco_Unitario: double (nullable = true)
 |-- Valor_Total: double (nullable = true)


Esquema do DataFrame de clientes:
root
 |-- cidade: string (nullable = true)
 |-- idade: long (nullable = true)
 |-- nome: string (nullable = true)

Dados de vendas:
+--------+----------+------------+-----------+----------+--------------+-----------+
|ID_Venda|Data_Venda|     Cliente|    Produto|Quantidade|Preco_Unitario|Valor_Total|
+--------+----------+------------+-----------+----------+--------------+-----------+
|       1|2024-07-01|  João Silva|   Camiseta|         2|          25.0|       50.0|
|       2|2024-07-01| Maria Souza|Calça Jeans|         1|          80.0|       80.0|
|       3|2024-07-02|Pedro Santos|      Tênis|         1|         150.0|      15

In [ ]:
#Filtragem 
#Filtrar vendas com quantidade maior que 3
vendas_filtradas = df_vendas.filter(col("quantidade") > 3)
print("Vendas com quantidade maior que 3:")
vendas_filtradas.show()

#Filtrar clientes de São Paulo
clientes_sp = df_clientes.filter(col("cidade") == "São Paulo")
print("\nClientes de São Paulo")
clientes_sp.show()

#Agregação
#Calcular o valor total das vendas
valor_total = df_vendas.agg(sum("Preco_Unitario").alias("valor_total"))
print("Valor total das vendas:")
valor_total.show()

#Calcular a média de idade dos clientes 
idade_media = df_clientes.agg(avg("idade").alias("idade_media"))
print("\nIdade média dos clientes:")
idade_media.show()

#União
#Criar um novo DataFrame com informações adicionais de vendas
df_vendas_adicionais = spark.createDataFrame([
    (6,"Monitor",2,800.00),
    (7,"Teclado",8,100.00)],["ID_Venda","produto","quantidade","Preco_Unitario"])

#Unir os DataFrames de vendas
df_vendas_uniao = df_vendas.union(df_vendas_adicionais)
print("União dos DataFrames de vendas:")
df_vendas_adicionais.show()
#Escrita de dados:

#Escrever o DataFrame de vendas unificado em um arquivo CSV
df_vendas_uniao.write.csv("vendas_unificadas.csv", header=True, mode="overwrite")

print("Arquivos exportados com sucesso!")
#Encerramento da sessão Spark:
#Encerrar a sessão Spark
spark.stop()


Vendas com quantidade maior que 3:
+--------+----------+-------+-------+----------+--------------+-----------+
|ID_Venda|Data_Venda|Cliente|Produto|Quantidade|Preco_Unitario|Valor_Total|
+--------+----------+-------+-------+----------+--------------+-----------+
+--------+----------+-------+-------+----------+--------------+-----------+


Clientes de São Paulo
+---------+-----+-------------+
|   cidade|idade|         nome|
+---------+-----+-------------+
|São Paulo|   30|João da Silva|
+---------+-----+-------------+

Valor total das vendas:
+-----------+
|valor_total|
+-----------+
|      265.0|
+-----------+


Idade média dos clientes:
+-----------+
|idade_media|
+-----------+
|       30.0|
+-----------+



AnalysisException: [NUM_COLUMNS_MISMATCH] UNION can only be performed on inputs with the same number of columns, but the first input has 7 columns and the second input has 4 columns. SQLSTATE: 42826;
'Union false, false
:- Relation [ID_Venda#430,Data_Venda#431,Cliente#432,Produto#433,Quantidade#434,Preco_Unitario#435,Valor_Total#436] csv
+- LogicalRDD [id#832L, produto#833, quantidade#834L, Preco_Unitario#835], false
